<a href="https://colab.research.google.com/github/Snake-AID/SnakeAid.ComputerVision/blob/main/SnakeTraining_YOLOv12_Khiem_v1plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive & cài thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Set thư mục chứa data.yaml + 3 folder train/test/valid

In [ ]:
DATASET_ROOT = "/content/drive/MyDrive/SnakeDataset/SnakeAid-YOLOv12-Masking-300img"

In [ ]:
!pip install -q -U ultralytics
print("✅ ultralytics installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:00
✅ ultralytics installed


# Validate Dataset

In [ ]:
from rich import print
from rich.panel import Panel
from rich.table import Table
from rich import box
import os

def check_dataset(root, folders=("train", "valid", "test")):
    table = Table(
        title="Dataset Structure",
        box=box.SIMPLE_HEAD,
        show_lines=False,
        pad_edge=False
    )

    table.add_column("Folder", style="bold cyan", justify="center")
    table.add_column("Images", justify="center")
    table.add_column("Labels", justify="center")

    for folder in folders:
        fp = os.path.join(root, folder)
        img = os.path.join(fp, "images")
        lbl = os.path.join(fp, "labels")

        table.add_row(
            folder.upper(),
            "[green]FOUND" if os.path.exists(img) else "[red]MISSING",
            "[green]FOUND" if os.path.exists(lbl) else "[red]MISSING",
        )

    panel = Panel.fit(
        table,
        title="🐍 Snake Dataset Check",
        border_style="cyan",
        box=box.SIMPLE
    )

    print(panel)

check_dataset(DATASET_ROOT)

    🐍 Snake Dataset Check    
      Dataset Structure       
                              
   Folder   Images   Labels   
   ────────────────────────   
   TRAIN    FOUND    FOUND    
   VALID    FOUND    FOUND    
    TEST    FOUND    FOUND    
                              
                              

#### Chọn kích thước model: `yolo12n.pt`, `yolo12s.pt`, `yolo12m.pt`, `yolo12l.pt`, `yolo12x.pt`

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo12nm.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Bắt đầu quá trình Train

In [ ]:
from rich import print
from rich.panel import Panel
from rich.table import Table
import logging

logging.getLogger("ultralytics").setLevel(logging.INFO)

def pretty_epoch_log(trainer):
    ep = trainer.epoch + 1
    n_ep = trainer.epochs

    # ---- Loss (safe & compatible) ----
    loss_items = getattr(trainer, "loss_items", None)

    if loss_items is not None:
        # loss_items có thể là Tensor hoặc list/tuple
        try:
            # Tensor: shape [3] (box, cls, dfl, ...)
            tloss = float(loss_items.sum().item())
        except AttributeError:
            # list/tuple: [box, cls, dfl, ...]
            tloss = float(sum(loss_items))
    else:
        tloss = float(getattr(trainer, "tloss", 0.0))

    # ---- Metrics (safe) ----
    m = getattr(trainer, "metrics", {}) or {}
    map50   = float(m.get("metrics/mAP50(B)")    or m.get("metrics/mAP50")    or 0.0)
    map5095 = float(m.get("metrics/mAP50-95(B)") or m.get("metrics/mAP50-95") or 0.0)

    table = Table(box=None, show_header=False, pad_edge=False)
    table.add_row("Epoch",      f"{ep}/{n_ep}")
    table.add_row("Train loss", f"{tloss:.4f}")
    table.add_row("mAP50",      f"{map50:.4f}")
    table.add_row("mAP50-95",   f"{map5095:.4f}")

    panel = Panel.fit(table, title="📊 Epoch summary", border_style="cyan")
    print(panel)

model.add_callback("on_fit_epoch_end", pretty_epoch_log)

results = model.train(
    data=os.path.join(DATASET_ROOT, "data.yaml"),
    epochs=17,
    imgsz=640,
    batch=16,
    verbose=True,
    val=True,
)


Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/SnakeDataset/SnakeAid-YOLOv12-Masking-300img/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=17, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True

╭─ 📊 Epoch summary ─╮
│ Epoch       1/17   │
│ Train loss  4.3157 │
│ mAP50       0.1398 │
│ mAP50-95    0.0534 │
╰────────────────────╯


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/17         0G     0.8598      2.464      1.227         11        640: 100% ━━━━━━━━━━━━ 20/20 19.0s/it 6:21
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 10.0s/it 10.0s
                   all         29         30    0.00578          1      0.132     0.0663


╭─ 📊 Epoch summary ─╮
│ Epoch       2/17   │
│ Train loss  5.3410 │
│ mAP50       0.1320 │
│ mAP50-95    0.0663 │
╰────────────────────╯


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/17         0G     0.8929      2.186      1.281         15        640: 100% ━━━━━━━━━━━━ 20/20 18.5s/it 6:11
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 10.8s/it 10.8s
                   all         29         30      0.554      0.147      0.237     0.0904


...

╭─ 📊 Epoch summary ─╮
│ Epoch       16/17  │
│ Train loss  2.5365 │
│ mAP50       0.5406 │
│ mAP50-95    0.4169 │
╰────────────────────╯


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      17/17         0G     0.4591      1.088      1.044          5        640: 100% ━━━━━━━━━━━━ 20/20 18.4s/it 6:08
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 10.9s/it 10.9s
                   all         29         30      0.839      0.615      0.559      0.388


╭─ 📊 Epoch summary ─╮
│ Epoch       17/17  │
│ Train loss  2.3861 │
│ mAP50       0.5595 │
│ mAP50-95    0.3876 │
╰────────────────────╯


17 epochs completed in 1.827 hours.
Optimizer stripped from /content/runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from /content/runs/detect/train/weights/best.pt, 5.5MB

Validating /content/runs/detect/train/weights/best.pt...
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLOv12n summary (fused): 159 layers, 2,557,508 parameters, 0 gradients, 6.3 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 10.0s/it 10.0s
                   all         29         30       0.81      0.562       0.54      0.417
              cap_nong          1          1          1          0          0          0
          ho_mang_xiem         21         21      0.801       0.81      0.757      0.596
   ran_hoa_can_van_dom          8          8       0.63      0.875      0.864      0.654
Speed: 2.8ms preprocess, 323.4ms inference, 0.0ms loss, 6.9ms postprocess per image
Results saved to /co

╭─ 📊 Epoch summary ─╮
│ Epoch       17/17  │
│ Train loss  2.3861 │
│ mAP50       0.5405 │
│ mAP50-95    0.4168 │
╰────────────────────╯

# Save file model sau khi Train vào Drive

In [ ]:
import os
import shutil

# đường dẫn file best.pt sau khi train (đã có sẵn)
model_source_path = "/content/runs_yolo12/exp_snake/weights/best.pt"

# thư mục lưu trên Drive
EXPORT_DIR = "/content/drive/MyDrive"

# Lấy tên file (best.pt)
exported_filename = os.path.basename(model_source_path)

# Xây dựng đường dẫn đích đầy đủ trong Google Drive
destination_path = os.path.join(EXPORT_DIR, exported_filename)

# Kiểm tra xem file best.pt có tồn tại không trước khi sao chép
if os.path.exists(model_source_path):
    # Sao chép file best.pt đã train đến thư mục trên Google Drive
    shutil.copy(model_source_path, destination_path)
    print(f"Mô hình best.pt đã được sao chép vào Drive tại: {destination_path}")
else:
    print(f"Lỗi: Không tìm thấy file {model_source_path}. Vui lòng đảm bảo quá trình training đã hoàn tất thành công.")

Lỗi: Không tìm thấy file /content/runs_yolo12/exp_snake/weights/best.pt. Vui lòng đảm bảo quá trình training đã 
hoàn tất thành công.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Đường dẫn đến thư mục kết quả huấn luyện
# (Giả định 'results' là biến từ quá trình train hoặc bạn biết đường dẫn)
# Trong trường hợp này, dựa vào output training, thư mục là /content/runs_yolo12/exp_snake
results_dir = "/content/runs_yolo12/exp_snake"
results_csv_path = os.path.join(results_dir, "results.csv")

# Kiểm tra xem file results.csv có tồn tại không
if os.path.exists(results_csv_path):
    # Đọc file CSV vào DataFrame
    df = pd.read_csv(results_csv_path)

    # Lấy các cột cần thiết
    epochs = df['epoch']
    # accuracy metrics thường là metrics/mAP50(B) và metrics/mAP50-95(B)
    # Tên cột có thể khác tùy phiên bản Ultralytics, bạn có thể kiểm tra df.columns
    mAP50 = df['metrics/mAP50(B)']
    mAP50_95 = df['metrics/mAP50-95(B)']

    # Vẽ biểu đồ
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, mAP50, label='mAP50')
    plt.plot(epochs, mAP50_95, label='mAP50-95')

    plt.title('Model Accuracy (mAP) over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('mAP Value')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print(f"Lỗi: Không tìm thấy file results.csv tại: {results_csv_path}. Vui lòng đảm bảo quá trình training đã hoàn tất thành công và tạo ra file này.")


Lỗi: Không tìm thấy file results.csv tại: /content/runs_yolo12/exp_snake/results.csv. Vui lòng đảm bảo quá trình 
training đã hoàn tất thành công và tạo ra file này.

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

# Đường dẫn đến mô hình tốt nhất đã train
model_path = "/content/runs_yolo12/exp_snake/weights/best.pt"

# Đường dẫn đến file data.yaml của bạn
data_yaml_path = os.path.join(DATASET_ROOT, "data.yaml")

# Tải mô hình
model = YOLO(model_path)

print("\n--- Đánh giá mô hình trên tập Test ---")

# Chạy đánh giá trên tập test
metrics = model.val(data=data_yaml_path, split='test')

# Lấy các chỉ số mAP
mAP50_test = metrics.results_dict['metrics/mAP50(B)']
mAP50_95_test = metrics.results_dict['metrics/mAP50-95(B)']

print(f"\nmAP50 trên tập Test: {mAP50_test:.4f}")
print(f"mAP50-95 trên tập Test: {mAP50_95_test:.4f}")

# Vẽ biểu đồ trực quan
labels = ['mAP50', 'mAP50-95']
values = [mAP50_test, mAP50_95_test]

plt.figure(figsize=(8, 6))
plt.bar(labels, values, color=['skyblue', 'lightcoral'])
plt.ylim(0, 1) # mAP values range from 0 to 1
plt.title('Model Accuracy on Test Set')
plt.ylabel('mAP Value')

for i, v in enumerate(values):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center', va='bottom')

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/runs_yolo12/exp_snake/weights/best.pt'

### 3. Sử dụng mô hình đã huấn luyện để dự đoán

In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow # Sử dụng để hiển thị ảnh trong Colab
import os

# Đường dẫn đến mô hình tốt nhất đã train và lưu trên Drive
model_path = "/content/runs_yolo12/exp_snake/weights/best.pt"
model = YOLO(model_path)

# --- Chuẩn bị ảnh đầu vào ---
# Thay thế bằng đường dẫn đến ảnh rắn mà bạn muốn kiểm tra.
input_image_path = "/content/638753879093007964.png"

# Kiểm tra xem file ảnh có tồn tại không
if not os.path.exists(input_image_path):
    print(f"Lỗi: Không tìm thấy file ảnh tại: {input_image_path}")
    print("Vui lòng thay thế 'input_image_path' bằng đường dẫn hợp lệ của ảnh bạn muốn dự đoán.")
else:
    print(f"\n--- Dự đoán trên ảnh: {input_image_path} ---")

    # Thực hiện dự đoán trên ảnh
    results = model.predict(source=input_image_path, conf=0.25) # conf: ngưỡng độ tin cậy tối thiểu để hiển thị box

    # Hiển thị kết quả `results[0].plot()` sẽ vẽ các hộp giới hạn và nhãn lên ảnh
    annotated_frame = results[0].plot()

    # Hiển thị ảnh đã được dự đoán
    cv2_imshow(annotated_frame)

    # Tùy chọn: Lưu ảnh kết quả
    # output_image_path = "/content/drive/MyDrive/predicted_snake_image.jpg"
    # cv2.imwrite(output_image_path, annotated_frame)
    # print(f"Ảnh kết quả đã được lưu tại: {output_image_path}")


In [ ]:
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow # Sử dụng để hiển thị ảnh trong Colab
import os
from tqdm import tqdm # Đã thay đổi: Import tqdm tiêu chuẩn thay vì tqdm.notebook

# Đường dẫn đến mô hình tốt nhất đã train (từ thư mục output của Ultralytics trong Colab)
model_path = "/content/runs_yolo12/exp_snake/weights/best.pt"

# Tải mô hình
model = YOLO(model_path)

# --- Cấu hình thư mục chứa ảnh và số lượng ảnh ---
images_directory = "/content/drive/MyDrive/SnakeTraining_YOLOv10_Khiem_v1/demo" # Thư mục chứa ảnh của bạn
num_images = 10
supported_extensions = ['.jpg', '.png', '.webp', '.jpeg'] # Thêm các định dạng file bạn muốn hỗ trợ

# Sử dụng tqdm để hiển thị thanh tiến trình
for i in tqdm(range(1, num_images + 1), desc="Processing images"):
    base_image_name = f"anh_{i}"
    input_image_path = None
    found_file = False

    for ext in supported_extensions:
        potential_path = os.path.join(images_directory, base_image_name + ext)
        if os.path.exists(potential_path):
            input_image_path = potential_path
            image_name = base_image_name + ext
            found_file = True
            break

    print(f"\n{'='*50}\n") # Dòng phân cách đẹp mắt

    if not found_file:
        print(f"Lỗi: Không tìm thấy file ảnh {base_image_name} với các định dạng hỗ trợ tại: {images_directory}. Bỏ qua ảnh này.")
        print(f"{'='*50}\n")
        continue # Bỏ qua ảnh này và chuyển sang ảnh tiếp theo
    else:
        print(f"Dự đoán trên ảnh: {image_name} ({input_image_path})")
        print(f"{'='*50}")

        # Thực hiện dự đoán trên ảnh
        # verbose=False để tắt output mặc định của Ultralytics và in log tùy chỉnh
        results = model.predict(source=input_image_path, conf=0.25, verbose=False)

        # Lấy kết quả của ảnh đầu tiên (và duy nhất) trong batch
        result = results[0]

        # Kiểm tra xem có đối tượng nào được phát hiện không
        if len(result.boxes) == 0:
            print("Không thấy rắn trong hình.") # Đã cập nhật thông báo
        else:
            print("Các đối tượng rắn được phát hiện:")
            for box in result.boxes:
                class_id = int(box.cls)
                class_name = result.names[class_id]
                confidence = float(box.conf)
                print(f"  - Loại: {class_name}, Độ tin cậy: {confidence:.2f}")

        # In thông tin tốc độ xử lý
        print("\nThông tin tốc độ:")
        for k, v in result.speed.items():
            print(f"  - {k.capitalize()}: {v:.1f}ms")

        # Hiển thị kết quả trực quan (sẽ hiển thị ảnh gốc nếu không có detections)
        annotated_frame = result.plot()

        # Hiển thị ảnh đã được dự đoán
        cv2_imshow(annotated_frame)

        print(f"{'='*50}\n") # Dòng phân cách đẹp mắt

        # Tùy chọn: Lưu ảnh kết quả
        # output_image_path = os.path.join(images_directory, f"predicted_{image_name}")
        # cv2.imwrite(output_image_path, annotated_frame)
        # print(f"Ảnh kết quả đã được lưu tại: {output_image_path}")


# Call tắt laptop sau khi train xong

In [ ]:
import requests

WEBHOOK_URL = "https://forcepslike-lawanda-semicalcined.ngrok-free.dev/done"  # đổi URL của bạn

try:
    r = requests.post(WEBHOOK_URL)
    print("Đã gửi tín hiệu shutdown về PC!")
except Exception as e:
    print("Không gửi được webhook:", e)